# Practical Application III: Comparing Classifiers

**Overview**: In this practical application, your goal is to compare the performance of the classifiers we encountered in this section, namely K Nearest Neighbor, Logistic Regression, Decision Trees, and Support Vector Machines.  We will utilize a dataset related to marketing bank products over the telephone.  



### Getting Started

Our dataset comes from the UCI Machine Learning repository [link](https://archive.ics.uci.edu/ml/datasets/bank+marketing).  The data is from a Portugese banking institution and is a collection of the results of multiple marketing campaigns.  We will make use of the article accompanying the dataset [here](CRISP-DM-BANK.pdf) for more information on the data and features.



### Problem 1: Understanding the Data

To gain a better understanding of the data, please read the information provided in the UCI link above, and examine the **Materials and Methods** section of the paper.  How many marketing campaigns does this data represent?

In [ ]:
The data represents 17 campaigns from May 2008 to November 2010.

### Problem 2: Read in the Data

Use pandas to read in the dataset `bank-additional-full.csv` and assign to a meaningful variable name.

In [2]:
%pip install scikit-learn
%pip install numpy
%pip install seaborn
%pip install matplotlib

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import time
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder, PolynomialFeatures
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression, RidgeClassifier, Lasso
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
import numpy as np
import seaborn as sns
import matplotlib.pylot as plt

ModuleNotFoundError: No module named 'matplotlib.pylot'

In [ ]:
bank_full_df = pd.read_csv('./data/bank-additional-full.csv', sep = ';')

In [215]:
bank_full_df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


### Problem 3: Understanding the Features


Examine the data description below, and determine if any of the features are missing values or need to be coerced to a different data type.


```
Input variables:
# bank client data:
1 - age (numeric)
2 - job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
3 - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
4 - education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')
5 - default: has credit in default? (categorical: 'no','yes','unknown')
6 - housing: has housing loan? (categorical: 'no','yes','unknown')
7 - loan: has personal loan? (categorical: 'no','yes','unknown')
# related with the last contact of the current campaign:
8 - contact: contact communication type (categorical: 'cellular','telephone')
9 - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
10 - day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
11 - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
# other attributes:
12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
14 - previous: number of contacts performed before this campaign and for this client (numeric)
15 - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')
# social and economic context attributes
16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)
17 - cons.price.idx: consumer price index - monthly indicator (numeric)
18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)
19 - euribor3m: euribor 3 month rate - daily indicator (numeric)
20 - nr.employed: number of employees - quarterly indicator (numeric)

Output variable (desired target):
21 - y - has the client subscribed a term deposit? (binary: 'yes','no')
```



In [217]:
# Number of rows with "unknown" value
num_unknown_job = (bank_full_df['job'] == 'unknown').sum() # 330
print('num_unknown_job', num_unknown_job)

num_unknown_marital = (bank_full_df['marital'] == 'unknown').sum() # 80
print('num_unknown_marital', num_unknown_marital)

num_unknown_education = (bank_full_df['education'] == 'unknown').sum() # 1731
print('num_unknown_education', num_unknown_education)

num_unknown_default = (bank_full_df['default'] == 'unknown').sum() # 8597
print('num_unknown_default', num_unknown_default)

num_unknown_housing = (bank_full_df['housing'] == 'unknown').sum() # 990
print('num_unknown_housing', num_unknown_housing)

num_unknown_loan = (bank_full_df['loan'] == 'unknown').sum() # 990
print('num_unknown_loan', num_unknown_loan)

# Rows with missing values
rows_with_missing_data = bank_full_df[bank_full_df.isnull().any(axis=1)]
print('rows_with_missing_data', rows_with_missing_data) # No rows are missing data


num_unknown_job 330
num_unknown_marital 80
num_unknown_education 1731
num_unknown_default 8597
num_unknown_housing 990
num_unknown_loan 990
rows_with_missing_data Empty DataFrame
Columns: [age, job, marital, education, default, housing, loan, contact, month, day_of_week, duration, campaign, pdays, previous, poutcome, emp.var.rate, cons.price.idx, cons.conf.idx, euribor3m, nr.employed, y]
Index: []

[0 rows x 21 columns]


In [218]:
# Convert 'month' column to one of four seasons 'winter' (Dec, Jan, Feb), 'spring', (March, April, May), 'summer' (June, July, Aug), 'fall' (Sept, Oct, Nov)
def categorize_season(month):
    if month in ['dec', 'jan', 'feb']:
        return 'winter'
    elif month in ['mar', 'apr', 'may']:
        return 'spring'
    elif month in ['jun', 'jul', 'aug']:
        return 'summer'
    else:
        return 'fall'

def categorize_age(age):
    if age < 30:
        return '<30'
    elif age >= 30 and age < 50:
        return '30-49'
    elif age >= 50 and age < 70:
        return '50-69'
    elif age >= 70 and age < 90:
        return '70-89'
    else:
        return '>90'

def categorize_pdays(pdays):
    if pdays >= 0 and pdays < 5:
        return '0-4'
    elif pdays >= 5 and pdays < 10:
        return '5-9'
    elif pdays >= 10 and pdays < 15:
        return '10-14'
    elif pdays >= 15 and pdays < 20:
        return '15-19'
    elif pdays >= 20 and pdays < 25:
        return '20-24'
    elif pdays >= 25 and pdays < 30:
        return '25-29'
    else:
        return 'not_prev_contacted'


def clean_df_data(df):
    # Remove rows with "unknown" value
    clean_df = df[df['job'] != 'unknown']
    clean_df = clean_df[clean_df['marital'] != 'unknown']
    clean_df = clean_df[clean_df['education'] != 'unknown']
    clean_df = clean_df[clean_df['default'] != 'unknown']
    clean_df = clean_df[clean_df['housing'] != 'unknown']
    clean_df = clean_df[clean_df['loan'] != 'unknown']

    # Remove "duration" column based on description above for better predictive model
    clean_df = clean_df.drop('duration', axis=1)

    # convert month to season
    clean_df['season'] = clean_df['month'].apply(categorize_season)
    # Drop month because it's now represented as season
    clean_df = clean_df.drop('month', axis=1)
    
    # convert pdays to pdays_range. Initial values from whole dataset 0 - 27 and 999
    clean_df['pdays_range'] = clean_df['pdays'].apply(categorize_pdays)
    clean_df['pdays_range'].unique()
    # Drop pdays because it's now represented as pdays_range
    clean_df = clean_df.drop('pdays', axis=1)

    # convert age to range. Initial values from whole dataset 17 - 95
    clean_df['age_range'] = clean_df['age'].apply(categorize_age)
    # Drop age because it's now represented as age_range
    clean_df = clean_df.drop('age', axis=1)
    return clean_df


### Problem 4: Understanding the Task

After examining the description and data, your goal now is to clearly state the *Business Objective* of the task.  State the objective below.

In [119]:
The Business Object is to analyze the data from 17 marketing campaigns to identify key factors
that influence client decisions, and create a predictive model that accurately predicts whether a
prospective client will subscribe to a term deposit. The bank will be able to use this model and 
information to create effective and efficient campaign strategies that increase subscriptions to term deposits.

### Problem 5: Engineering Features

Now that you understand your business objective, we will build a basic model to get started.  Before we can do this, we must work to encode the data.  Using just the bank information features, prepare the features and target column for modeling with appropriate encoding and transformations.

In [150]:
clean_bank_full_df=clean_df_data(bank_full_df)

X = clean_bank_full_df.drop('y', axis=1)
y = clean_bank_full_df.y

le = LabelEncoder()
y = le.fit_transform(y)

categorical_columns = X.select_dtypes(include=['object']).columns.tolist()
numerical_columns = X.select_dtypes(include=['float64', 'int64']).columns.tolist()
print('categorical_columns', categorical_columns)
print('numerical_columns', numerical_columns)

categorical_columns ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'day_of_week', 'poutcome', 'season', 'pdays_range', 'age_range']
numerical_columns ['campaign', 'previous', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed']


In [151]:
# Encodes the data based on whether it's numerical or categorical
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_columns),
        ('cat', OneHotEncoder(drop='first'), categorical_columns)
    ]
)

### Problem 6: Train/Test Split

With your data prepared, split it into a train and test set.

In [152]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


### Problem 7: A Baseline Model

Before we build our first model, we want to establish a baseline.  What is the baseline performance that our classifier should aim to beat?

In [ ]:
The classifier should ideally accurately predict whether a client will subscribe to a term deposit 90% of the time.
This level of accuracy would provide the bank with significant power to target clients most likely to convert while 
recognizing that inaccurate estimates is not exorbinantly expensive either monetarily or consequentially.

### Problem 8: A Simple Model

Use Logistic Regression to build a basic model on your data.  

In [153]:
# Create a pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('logisticregression', LogisticRegression(max_iter=500))
])
    
# Perform grid search
grid_search = GridSearchCV(pipeline, param_grid={'logisticregression__C': [0.1, 1, 10]}, cv=5, n_jobs=-1, scoring="accuracy")

# Fit the model and time it
start_time = time.time()
grid_search.fit(X_train, y_train)
fit_time = (time.time() - start_time) / len(grid_search.cv_results_['mean_fit_time'])

# Get the best estimator
best_model = grid_search.best_estimator_

# Evaluate on training and test sets
train_score = best_model.score(X_train, y_train)
test_score = best_model.score(X_test, y_test)
print('fit_time', fit_time)
print('best_model', best_model)
print('train_score', train_score)
print('test_score', test_score)
logistic_regression_result={ "model": "Logistic Regression", "train_time": fit_time, "train_accuracy": train_score, "test_accuracy": test_score}

fit_time 1.5651466051737468
best_model Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['campaign', 'previous',
                                                   'emp.var.rate',
                                                   'cons.price.idx',
                                                   'cons.conf.idx', 'euribor3m',
                                                   'nr.employed']),
                                                 ('cat',
                                                  OneHotEncoder(drop='first'),
                                                  ['job', 'marital',
                                                   'education', 'default',
                                                   'housing', 'loan', 'contact',
                                                   'day_of_week', 'poutcome',
                                                   'se

### Problem 9: Score the Model

What is the accuracy of your model?

In [ ]:
The model was 88.68% accurate on the test data.

### Problem 10: Model Comparisons

Now, we aim to compare the performance of the Logistic Regression model to our KNN algorithm, Decision Tree, and SVM models.  Using the default settings for each of the models, fit and score each.  Also, be sure to compare the fit time of each of the models.  Present your findings in a `DataFrame` similar to that below:

| Model | Train Time | Train Accuracy | Test Accuracy |
| ----- | ---------- | -------------  | -----------   |
|     |    |.     |.     |

In [154]:
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('knn', KNeighborsClassifier())
])
    
# Perform grid search
grid_search = GridSearchCV(pipeline, param_grid={'knn__n_neighbors': [3, 5, 7]}, cv=5, n_jobs=-1, scoring="accuracy")

# Fit the model and time it
start_time = time.time()
grid_search.fit(X_train, y_train)
fit_time = (time.time() - start_time) / len(grid_search.cv_results_['mean_fit_time'])

# Get the best estimator
best_model = grid_search.best_estimator_

# Evaluate on training and test sets
train_score = best_model.score(X_train, y_train)
test_score = best_model.score(X_test, y_test)
print('fit_time', fit_time)
print('best_model', best_model)
print('train_score', train_score)
print('test_score', test_score)
knn_result={ "model": "KNN", "train_time": fit_time, "train_accuracy": train_score, "test_accuracy": test_score}

fit_time 14.103101968765259
best_model Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['campaign', 'previous',
                                                   'emp.var.rate',
                                                   'cons.price.idx',
                                                   'cons.conf.idx', 'euribor3m',
                                                   'nr.employed']),
                                                 ('cat',
                                                  OneHotEncoder(drop='first'),
                                                  ['job', 'marital',
                                                   'education', 'default',
                                                   'housing', 'loan', 'contact',
                                                   'day_of_week', 'poutcome',
                                                   'se

In [155]:
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('decisiontreeclassifier', DecisionTreeClassifier())
])
    
# Perform grid search
grid_search = GridSearchCV(pipeline, param_grid={'decisiontreeclassifier__max_depth': [2, 5, 7]}, cv=5, n_jobs=-1, scoring="accuracy")

# Fit the model and time it
start_time = time.time()
grid_search.fit(X_train, y_train)
fit_time = (time.time() - start_time) / len(grid_search.cv_results_['mean_fit_time'])

# Get the best estimator
best_model = grid_search.best_estimator_

# Evaluate on training and test sets
train_score = best_model.score(X_train, y_train)
test_score = best_model.score(X_test, y_test)
print('fit_time', fit_time)
print('best_model', best_model)
print('train_score', train_score)
print('test_score', test_score)
decision_tree_classifier_result={ "model": "Decision Tree Classifier", "train_time": fit_time, "train_accuracy": train_score, "test_accuracy": test_score}

fit_time 0.3018353780110677
best_model Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['campaign', 'previous',
                                                   'emp.var.rate',
                                                   'cons.price.idx',
                                                   'cons.conf.idx', 'euribor3m',
                                                   'nr.employed']),
                                                 ('cat',
                                                  OneHotEncoder(drop='first'),
                                                  ['job', 'marital',
                                                   'education', 'default',
                                                   'housing', 'loan', 'contact',
                                                   'day_of_week', 'poutcome',
                                                   'se

In [156]:
# Using the smaller data sample for the SVC because it's computationally demanding
bank_additional_df = pd.read_csv('./data/bank-additional.csv', sep = ';')

clean_bank_additional_df=clean_df_data(bank_additional_df)

X_additional = clean_bank_additional_df.drop('y', axis=1)
y_additional = clean_bank_additional_df.y

le = LabelEncoder()
y_additional = le.fit_transform(y_additional)
categorical_columns = X_additional.select_dtypes(include=['object']).columns.tolist()
numerical_columns = X_additional.select_dtypes(include=['float64', 'int64']).columns.tolist()

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_columns),
        ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), categorical_columns) # Ignoring unknown because this is a subset of the whole data
    ]
)
X_additional_train, X_additional_test, y_additional_train, y_additional_test = train_test_split(X_additional, y_additional, test_size=0.2, random_state=42)
categorical_columns

['job',
 'marital',
 'education',
 'default',
 'housing',
 'loan',
 'contact',
 'day_of_week',
 'poutcome',
 'season',
 'pdays_range',
 'age_range']

In [157]:
# SVC Processor
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('svc', SVC())
])
    
# Perform grid search
grid_search = GridSearchCV(pipeline, param_grid={'svc__C': [0.1, 1, 10], 'svc__kernel': ['linear', 'rbf']}, cv=5, n_jobs=-1, scoring="accuracy")

# Fit the model and time it
start_time = time.time()
grid_search.fit(X_additional_train, y_additional_train)
fit_time = (time.time() - start_time) / len(grid_search.cv_results_['mean_fit_time'])

# Get the best estimator
best_model = grid_search.best_estimator_

# Evaluate on training and test sets
train_score = best_model.score(X_additional_train, y_additional_train)
test_score = best_model.score(X_additional_test, y_additional_test)
print('fit_time', fit_time)
print('best_model', best_model)
print('train_score', train_score)
print('test_score', test_score)
svc_result={ "model": "SVC", "train_time": fit_time, "train_accuracy": train_score, "test_accuracy": test_score}

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [10] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [10] during transform. These unknown categor

fit_time 0.1742851734161377
best_model Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['campaign', 'previous',
                                                   'emp.var.rate',
                                                   'cons.price.idx',
                                                   'cons.conf.idx', 'euribor3m',
                                                   'nr.employed']),
                                                 ('cat',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore'),
                                                  ['job', 'marital',
                                                   'education', 'default',
                                                   'housing', 'loan', 'contact',
                                           

In [158]:
all_results=[logistic_regression_result, knn_result, decision_tree_classifier_result, svc_result]
result_df = pd.DataFrame(all_results)
result_df

,model,train_time,train_accuracy,test_accuracy
0,Logistic Regression,1.565147,0.889381,0.886848
1,KNN,14.103102,0.898729,0.875205
2,Decision Tree Classifier,0.301835,0.892005,0.885044
3,SVC,0.174285,0.886327,0.914239


### Problem 11: Improving the Model

Now that we have some basic models on the board, we want to try to improve these.  Below, we list a few things to explore in this pursuit.

- More feature engineering and exploration.  For example, should we keep the gender feature?  Why or why not?
- Hyperparameter tuning and grid search.  All of our models have additional hyperparameters to tune and explore.  For example the number of neighbors in KNN or the maximum depth of a Decision Tree.  
- Adjust your performance metric

In [195]:
# Explore the impact of different features
clean_all_df=clean_df_data(bank_full_df)

def add_default_prefix(word):
    return "default_" + word
def add_housing_prefix(word):
    return "housing_" + word
def add_loan_prefix(word):
    return "loan_" + word

clean_all_df['default'] = clean_all_df['default'].apply(add_default_prefix)
clean_all_df['housing'] = clean_all_df['housing'].apply(add_housing_prefix)
clean_all_df['loan'] = clean_all_df['loan'].apply(add_loan_prefix)


In [209]:
# Explore the impact of different features
dummies_job=pd.get_dummies(clean_all_df['job'], dtype=int)
dummies_marital=pd.get_dummies(clean_all_df['marital'], dtype=int)
dummies_education=pd.get_dummies(clean_all_df['education'], dtype=int)
dummies_default=pd.get_dummies(clean_all_df['default'], dtype=int)
dummies_housing=pd.get_dummies(clean_all_df['housing'], dtype=int)
dummies_loan=pd.get_dummies(clean_all_df['loan'], dtype=int)
dummies_contact=pd.get_dummies(clean_all_df['contact'], dtype=int)
dummies_day_of_week=pd.get_dummies(clean_all_df['day_of_week'], dtype=int)
dummies_poutcome=pd.get_dummies(clean_all_df['poutcome'], dtype=int)
dummies_season=pd.get_dummies(clean_all_df['season'], dtype=int)
dummies_pdays_range=pd.get_dummies(clean_all_df['pdays_range'], dtype=int)
dummies_age_range=pd.get_dummies(clean_all_df['age_range'], dtype=int)
df = pd.concat([dummies_job, dummies_marital, dummies_education, dummies_default, dummies_housing, dummies_loan, dummies_contact, dummies_day_of_week, dummies_poutcome, dummies_season, dummies_pdays_range, dummies_age_range], axis=1)
clean_all_df['y']=np.where(clean_all_df['y'] == 'no', 0, 1)
y=clean_all_df['y']
y.head(5)
X_lasso_train, X_lasso_test, y_lasso_train, y_lasso_test=train_test_split(df, y, test_size=0.2, random_state=42)

In [ ]:
# Explore the impact of different features
pipe = Pipeline([
    ('polyfeatures', PolynomialFeatures(degree = 3, include_bias = False)),
    ('scaler', StandardScaler()),
    ('lasso', Lasso(random_state=42))
])
pipe.fit(X_lasso_train, y_lasso_train)
lasso_coefs=pipe.named_steps['lasso'].coef_
feature_names=pipe.named_steps['polyfeatures'].get_feature_names_out()
lasso_df=pd.DataFrame({ 'feature': feature_names, 'coef': lasso_coefs })
results=lasso_df.loc[lasso_df['coef'] != 0]
sorted = results.sort_values(by='coef', ascending=False)
print(sorted.head(15))

##### Questions